# Context

Multi-stage preprocessing pipelines that include multiple nonlinear transformations are hard to understand. To increase transparency and make it easy for the reader to understand which signals are filtered, a minimal procedure is desirable that results in a representation where the signal of interestg shows. This allows to present results going from specific to general.

# Imports

In [32]:
from skimage import io
import skimage
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, uniform_filter
import pickle

In [33]:
import imageio
from pathlib import Path
from matplotlib.pyplot import show
from argparse import ArgumentParser

from pyoptflow import HornSchunck, getimgfiles
from pyoptflow.plots import compareGraphs

In [34]:
from PIL import Image
import os
from scipy.signal import argrelextrema
from skimage import exposure

In [35]:
import matplotlib
import matplotlib.animation
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [36]:
np.array(np.clip([300],0,255), dtype=np.uint8)

array([255], dtype=uint8)

### Import our custom utility methods

In [37]:
import sys
%reload_ext autoreload
%autoreload 2
sys.path.append('..')

from utils.visualization_tools import *
import utils.visualization_tools
from utils.data_transformations import *
import utils.data_transformations
from utils.diverse import *
import utils.diverse

The following modules are available

In [38]:
print_module_methods(utils.diverse)

print_module_methods(module)



In [39]:
print_module_methods(utils.visualization_tools)

display_combined(u, v, Inew, scale=100, quivstep=3, fig=None, ax=None, figsize=(10, 10), vmin=0, vmax=1)

print_points_and_background(img, x, y, point_size=10, marker='.')



In [40]:
print_module_methods(utils.data_transformations)

HornSchunck(im1: numpy.ndarray, im2: numpy.ndarray, *, alpha: float = 0.001, Niter: int = 8, verbose: bool = False) -> Tuple[numpy.ndarray, numpy.ndarray]

apply_mask(frames, mask)

argrelextrema(data, comparator, axis=0, order=1, mode='clip')

clipped_adaptive(tensor, clipping=0.8)

fourier(signal, sampling_rate=100)

framewise_difference(frames, pixelwise_mean, bigdata=False)

gaussian_filter(input, sigma, order=0, output=None, mode='reflect', cval=0.0, truncate=4.0)

getimgfiles(stem: pathlib.Path, pat: str) -> list

horn_schunck(tensor, frames=None)

maxima(vector, pre_smoothing=100, minval=0)

normalize(frames)

poly_smooth_2d(coords, polynomial_degree=11, steps_per_pixel=10, epsilon=0.1)

substract_pixel_min(tensor)

uniform_filter(input, size=3, output=None, mode='reflect', cval=0.0, origin=0)



In [48]:
def sample_frame_in_roi(frame, window_size, left, right, top, bottom):   
    further_preprocessed = exposure.equalize_adapthist(normalize(frame[left:right,top:bottom]), clip_limit=0.03)
    further_preprocessed = further_preprocessed[:window_size-8,:window_size-8]
    return further_preprocessed
    
def sample_roi(tensor, start_frame, stop_frame, window_size = 60,left = 120, top = 80,):
    right = left + window_size
    bottom = top + window_size
    return np.array([sample_frame_in_roi(tensor[i], window_size, left, right, top, bottom) for i in range(start_frame,stop_frame)])

# Retrieve blobs with a minimal number of transformation

In [41]:
from pathlib import Path
source_folder = os.path.join(Path(os.getcwd()).parent, "source_data")

In [51]:
#Load data and select ROI
frames = skimage.io.imread(os.path.join(source_folder,"runstart16_X1.tif"))
frames = frames[:1000,:,:]
frames = sample_roi(frames,0,1000)

# Preprocessing

In [122]:
# Compute difference to mean
mean = np.mean(frames,axis=0)#pixelwise mean
difference = gaussian_filter(framewise_difference(frames, mean, bigdata=False),1)
details = normalize(np.array([normalize(frame) for frame in difference]))

In [137]:
%%capture
fig, ax = plt.subplots(1, figsize=(10,10))

im = ax.imshow(details[0,:,:], vmin =.0, vmax=1)#vmin=.25,vmax=.3)
startframe = 70
ani = matplotlib.animation.FuncAnimation(fig, lambda i: im.set_array(details[startframe+i]), frames=100).to_jshtml()

In [138]:
HTML(ani)

# Conclusion



The details that show moving clusters of activation are visible after applying only few transformations if a small region of interest is selected. These steps are:

- Background substraction
- Slight smoothing to increase the signal to noise ratio
- Framewise normalization to remove the global trend due to changes in overall brightness

This procedure has several drawbacks.

- Low contrast
- Due to noise the maximal and minimal value between frames jumps up and down randomly. As framewise normalization is based upon these values 
- It is only applicable for small regions of interest where the assumption that the global trend in activation is uniformly distributed in space is justified. By framewise normalization one only considers the global trend in the time dimension not in the spacial dimension.

Hence a more general yet more complex procedure must be used.